In [ ]:
import polars as pl
from datetime import datetime

In [ ]:
#Question 1
from google.colab import drive
drive.mount('/content/drive')

file_path = "/content/drive/MyDrive/Colab Notebooks/faa_strikes.txt"



strikes = pl.read_csv(
    file_path,
    separator="\t",
    has_header=True,
    infer_schema_length=10000,
    null_values=["", "NA"],
    truncate_ragged_lines=True
)

print(strikes.shape)
print(strikes.columns)
strikes.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(28298, 25)
['Airport: Code', 'Airport: Name', 'Origin State', 'Origin State Code', 'Country', 'Aircraft: Type', 'Aircraft: Number of engines', 'Collision Date and Time', 'When: Time of day', 'When: Phase of flight', 'Effect: Amount of damage (detailed)', 'Effect: Impact to flight', 'Effect: Indicated Damage', 'Cost: Aircraft time out of service (hours)', 'Cost: Total $', 'Days', 'Feet above ground', 'Miles from airport', 'Wildlife: Animal Category', 'Wildlife: Species Order', 'Wildlife: Species Group', 'Wildlife: Species', 'Wildlife: Species ID', 'Number of Strikes', 'Record ID']


Airport: Code,Airport: Name,Origin State,Origin State Code,Country,Aircraft: Type,Aircraft: Number of engines,Collision Date and Time,When: Time of day,When: Phase of flight,Effect: Amount of damage (detailed),Effect: Impact to flight,Effect: Indicated Damage,Cost: Aircraft time out of service (hours),Cost: Total $,Days,Feet above ground,Miles from airport,Wildlife: Animal Category,Wildlife: Species Order,Wildlife: Species Group,Wildlife: Species,Wildlife: Species ID,Number of Strikes,Record ID
str,str,str,str,str,str,i64,str,str,str,str,str,str,i64,i64,i64,i64,i64,str,str,str,str,str,i64,i64
"""KAAF""","""APALACHICOLA REGIONAL ARPT""","""Florida""","""FL""","""United States""",null,null,"""2012-09-20T23:30:00Z""",null,"""Take-off run""","""None""","""None""","""No damage""",null,0,null,0,null,"""Terrestrial Mammals""","""Canids""","""Wolves, Dogs, Foxes, Coyote""","""Domestic dog""","""1F12""",1,17459
"""KAAF""","""APALACHICOLA REGIONAL ARPT""","""Florida""","""FL""","""United States""","""Airplane""",1,"""2013-04-23T21:09:00Z""",null,"""Take-off run""","""None""","""None""","""No damage""",null,0,null,null,null,"""Birds""","""Pelicans, Herons, Egrets, Bitt…","""Herons, Egrets, Bitterns""","""Herons, egrets, bitterns""","""I1""",1,17114
"""KABE""","""LEHIGH VALLEY INTL""","""Pennsylvania""","""PA""","""United States""","""Airplane""",2,"""2009-04-23T13:22:00Z""","""Day""","""Take-off run""","""Medium""","""Aborted Take-off""","""Caused damage""",720,171132,30,0,0,"""Birds""","""Caracaras, Falcons""","""Caracaras, Falcons""","""American kestrel""","""K5114""",1,259361
"""KABE""","""LEHIGH VALLEY INTL""","""Pennsylvania""","""PA""","""United States""","""Airplane""",2,"""2014-04-14T02:00:00Z""","""Night""","""Approach""","""None""","""None""","""No damage""",15,600,1,null,null,"""Birds""","""Pigeons and Doves""","""Doves""","""Mourning dove""","""O2205""",1,345167
"""KABE""","""LEHIGH VALLEY INTL""","""Pennsylvania""","""PA""","""United States""","""Airplane""",2,"""2009-03-31T22:15:00Z""","""Day""","""Approach""","""Medium""","""None""","""Caused damage""",12,188245,1,null,0,"""Birds""","""Hawks, Kites, Eagles, Ospreys,…","""Kites, Hawks, Eagles""","""Red-tailed hawk""","""K3302""",1,262782


In [ ]:
#Question 2

strikes = strikes.rename({c: c.strip() for c in strikes.columns})

running_total_strikes = (
    strikes
    .with_columns(
        pl.col("Collision Date and Time")
          .str.strptime(pl.Datetime, strict=False)
          .dt.date()
          .alias("date")
    )
    .group_by("date")
    .agg(pl.col("Number of Strikes").sum().alias("daily_strikes"))
    .sort("date")
    .with_columns(
        pl.col("daily_strikes").cum_sum().alias("strikes_cumulative")
    )
    .filter(pl.col("date") <= pl.date(2013, 12, 31))
)

running_total_strikes

date,daily_strikes,strikes_cumulative
date,i64,i64
2000-01-02,1,1
2000-01-03,2,3
2000-01-05,3,6
2000-01-06,1,7
2000-01-08,4,11
…,…,…
2013-12-27,6,24096
2013-12-28,1,24097
2013-12-29,2,24099


In [ ]:
#Question 3
type_species = (
    strikes

    .group_by(["Aircraft: Type", "Wildlife: Species Group"])
    .agg(
        pl.col("Cost: Total $").sum().alias("damage")
    )

    .with_columns(
        pl.col("damage")
        .rank(method="dense", descending=True)
        .over("Aircraft: Type")
        .alias("ranking")
    )

    .filter(pl.col("ranking") == 2)
)

type_species

Aircraft: Type,Wildlife: Species Group,damage,ranking
str,str,i64,u32
"""Helicopter""","""Vultures""",116748,2
"""Airplane""","""Kites, Hawks, Eagles""",48989287,2
null,"""Vultures""",56227,2


In [ ]:
#Question 4

greatest_strike_increase = (
    strikes
    .with_columns(
        pl.col("Collision Date and Time")
          .str.strptime(pl.Datetime, strict=False)
          .dt.date()
          .alias("date")
    )
    .group_by("date")
    .agg(pl.col("Number of Strikes").sum().alias("daily_strikes"))
    .sort("date")
    .with_columns(pl.col("daily_strikes").shift(1).alias("previous_day"))
    .with_columns((pl.col("daily_strikes") - pl.col("previous_day")).alias("delta_strikes"))
    .sort("delta_strikes", descending=True)
)

greatest_strike_increase

date,daily_strikes,previous_day,delta_strikes
date,i64,i64,i64
2000-01-02,1,null,null
2010-10-29,25,7,18
2014-08-25,25,8,17
2012-06-28,21,5,16
2009-07-07,21,7,14
…,…,…,…
2012-06-29,6,21,-15
2012-10-14,5,20,-15
2014-08-26,10,25,-15


In [ ]:
#Question 5
strikes = strikes.rename({c: c.strip() for c in strikes.columns})

greatest_strike_increase_type = (
    strikes
    .with_columns(
        pl.col("Collision Date and Time")
          .str.strptime(pl.Datetime, strict=False)
          .dt.date()
          .alias("date")
    )
    .group_by(["Aircraft: Type", "date"])
    .agg(pl.col("Number of Strikes").sum().alias("daily_strikes"))
    .sort(["Aircraft: Type", "date"])
    .with_columns(
        pl.col("daily_strikes").shift(1).over("Aircraft: Type").alias("previous_day")
    )
    .with_columns(
        (pl.col("daily_strikes") - pl.col("previous_day")).alias("delta_strikes")
    )
    .with_columns(
        pl.col("delta_strikes")
          .rank(method="dense", descending=True)
          .over("Aircraft: Type")
          .alias("ranking")
    )
    .filter(pl.col("ranking") == 1)
)

greatest_strike_increase_type

Aircraft: Type,date,daily_strikes,previous_day,delta_strikes,ranking
str,date,i64,i64,i64,u32
null,2014-08-04,5,1,4,1
"""Airplane""",2014-08-25,25,8,17,1
"""Helicopter""",2010-07-21,2,1,1,1
"""Helicopter""",2012-11-02,2,1,1,1
"""Helicopter""",2013-10-28,2,1,1,1
"""Helicopter""",2014-08-14,2,1,1,1
